In [44]:
%%file deals.py


"""
>>> company = Company(symbol="tsm", sticker_price=11)
>>> get_deal(company, Ticker=MockTicker)
Deal("tsm", 11, 98)

>>> companies = [
...  Company(symbol="msft", sticker_price=118),
...  Company(symbol="tsm", sticker_price=11)
... ]
>>> deals = get_deals(companies, Ticker=MockTicker)
>>> deals
[Deal("msft", 118, 352.5), Deal("tsm", 11, 98)]
"""


from types import SimpleNamespace
from yfinance import Ticker


class Company(SimpleNamespace):
    def __init__(self, symbol, sticker_price):
        self.symbol = symbol
        self.sticker_price = sticker_price


class Deal:

    """
    >>> Deal("msft", 1, 1)
    Deal("msft", 1, 1)
    """

    def __init__(
        self, symbol, sticker_price,
        price
    ):
        self.symbol = symbol
        self.sticker_price = sticker_price
        self.price = price
        self.percent_of_sticker_price = price / sticker_price * 100

    def __repr__(self):
        return f'Deal("{self.symbol}", {self.sticker_price}, {self.price})'

    def __eq__(self, other):
        if self.sticker_price != other.sticker_price:
            return False

        if self.price != other.price:
            return False

        return True

class MockTicker:
    def __init__(self, symbol):
        price = 352.5 if symbol == "msft" else 98

        self.info = {
            "currentPrice": price
        }


def get_price(symbol, Ticker=Ticker):
    """
    >>> get_price("msft", Ticker=MockTicker)
    352.5
    >>> get_price([])
    {}
    >>> get_price(["msft"], Ticker=MockTicker)
    {'msft': 352.5}
    """
    if type(symbol) == str:
        price = Ticker(symbol).info["currentPrice"]
        return price

    prices = {
        the_symbol: get_price(the_symbol, Ticker=Ticker)
        for the_symbol in symbol
    }
    return prices

def get_deal(company, Ticker=Ticker):
    price = get_price(company.symbol, Ticker=Ticker)
    return Deal(company.symbol, company.sticker_price, price)

def get_deals(companies, Ticker=Ticker):
    sticker_prices = {
        company.symbol: company.sticker_price
        for company in companies
    }

    deals = []
    for symbol in sticker_prices.keys():
        price = get_price(symbol, Ticker=Ticker)
        deals.append(Deal(symbol, sticker_prices[symbol], price))
    
    return deals

def test_get_deals():
    companies = [
        Company("msft", 118),
        Company("tsm", 22)
    ]

    deals = get_deals(companies, Ticker=MockTicker)

    assert deals[0] == Deal("msft", 118, 352.5)

def test_get_deal_one():
    company = Company(symbol="msft", sticker_price=118)

    deal = get_deal(company, Ticker=MockTicker)

    assert deal == Deal("msft", 118, 352.5)

Overwriting deals.py


In [45]:
!pytest --doctest-modules deals.py

============================= test session starts ==============================
platform linux -- Python 3.10.11, pytest-7.2.2, pluggy-1.0.0
rootdir: /content
plugins: anyio-3.6.2
collected 5 items                                                              

deals.py .....                                                           [100%]

============================== 5 passed in 0.73s ===============================
